# Import a inicializace

In [1]:
#!/usr/bin/env python
import copy                                  # Kopirovani prvku
import numpy as np                           # Matematicke operace s maticemi
import random                                # Vyber nahodnych prvku
import h5py                                  # Ukladani vah site
import gym                                   # Prostredi Open AI Gym
#from gym import wrappers                    # Nahravani zaznamu
environment = "MountainCar-v0"               # Jmeno prostredi
env = gym.make(environment)                  # Konkretni hra z Open AI Gym
#env = wrappers.Monitor(env, '/home/lachubcz/tmp/cartpole-experiment-1', force=True)

%matplotlib inline
import matplotlib.pyplot as plt
from collections import deque                # Pamet
from __future__ import division              # Deleni realnych cisel (kvuli nizsi verzi Pythonu 2.6)
from tqdm import tnrange, tqdm_notebook      # Progress bar
from sources.profiling import * #profiling - @do_profile(follow=[method, ])
from sources.playing import *
from sources.visualization import *
from sources.agent import *

#Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics

#Tensorflow session
import tensorflow as tf                     # Knihovna Tensorflow
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
set_session(tf.Session(config=config))

# Parametry
observetime = 200                            # Delka pozorovani
episodes = 5                              # Pocet epizod
games = 25                                   # Pocet her
scores = []                                  # Pole pro ulozeni vysledku na analyzu
episodesList = []                            # Pole pro ulozeni cisel epizod na analyzu
bestScore = float("-inf")                    # Promenna pro ukladani nejlepsiho prubezneho vysledku
best_avg_score = float("-inf")               # Promenna pro ukladani nejlepsiho prumerneho vysledku
actionCount = env.action_space.n             # Pocet vstupu do prostredi
stateSize = env.observation_space.shape[0]   # Pocet vystupu z prostredi
agent = Agent(env)                           # Vytvoreni agenta

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                96        
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 51        
Total params: 675
Trainable params: 675
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                96        
_________________________________________________________________
dense_5 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_6 (Den

# Algoritmus

In [2]:
rand_agent_replay(env, agent, 10000, 500, actionCount)

#agent.loadNN("./DDQN-MountainCar-v0.h5")                                     # Nacteni vah NN
#agent.updateTargetNet()                                                      # Nacteni NN do netTarget

for eps in tnrange(episodes, desc='episodes'):
    state = env.reset()                                                   # Resetovani prostredi
    total_reward = 0                                                      # Vynulovani hodnoty za kolo
    
    for t in range(observetime):
        state = np.reshape(state, (1, stateSize))                         # Formatovani
        action = agent.getActionWE(state)                                 # Ziskani akce
        nextState, reward, done, info = env.step(action)                  # Provedeni akce

        agent.remember(state, action, reward, nextState, done)            # Ulozeni stavu do pameti
        agent.trainDQN()                                                  # Trenovani pameti
        
        state = nextState                                                 # Zmena stavu
        total_reward = total_reward + reward                              # Pricteni odmeny za aktualni stav
        
        agent.epsilonActulization()                                       # Aktualizace epsilon
        if done:                                                          # Konec epizody
            #agent.updateTargetNet()                                      # Aktualizace target site
            
            if eps % 50 == 0:
                avg_score = score_estimate(env, agent, games, stateSize)  # Vypocet aktualniho skore
                scores.append(avg_score)                                  # Ulozeni aktualniho skore
                episodesList.append(eps)                                  # Ulozeni aktualniho cisla epizody
                
                print("Episode: {}/{}, epsilon: {:.2}, average score: {}".format(eps, episodes, agent.currentEpsilon, avg_score))
                
                if avg_score > best_avg_score and eps != 0:
                    best_avg_score = avg_score
                    agent_replay(env, agent,  10000, 500, stateSize)
                
                agent.saveNN("./saves/{}-{}.h5" .format(environment, eps))      # Ulozeni site
                
            if total_reward > bestScore:                                  # Bylo dosazeno nejlepsiho skore
                bestScore = total_reward                                  # Nove nejlepsi skore
                print("Episode: {}/{}, epsilon: {:.2}, new best score: {}".format(eps, episodes, agent.currentEpsilon, total_reward))
            break

agent.saveNN("./saves/{}-DONE.h5" .format(environment, eps))                    # Ulozeni site

A Jupyter Widget

State: [[ -4.42544432e-01  -4.54735808e-04]
 [ -3.81539739e-01   3.35950476e-03]
 [ -4.52762619e-01   3.63390664e-03]
 [ -5.72064077e-01  -5.20137439e-03]
 [ -4.84627605e-01  -4.57273613e-04]
 [ -5.97443935e-01   7.62636548e-03]
 [ -4.41496416e-01  -1.39859007e-03]
 [ -5.51124981e-01  -1.64321878e-03]
 [ -5.50503996e-01   1.04028448e-02]
 [ -5.17239510e-01  -8.59671344e-03]
 [ -4.91268623e-01  -2.05773239e-03]
 [ -5.07302482e-01  -6.08342932e-03]
 [ -4.58819794e-01  -5.28065211e-05]
 [ -5.07456879e-01  -1.15648592e-02]
 [ -5.35163769e-01  -2.04188000e-03]
 [ -5.17393877e-01  -3.93046890e-03]
 [ -5.32673597e-01  -1.99511897e-03]
 [ -5.84639263e-01  -3.83596360e-03]
 [ -5.97204332e-01  -6.79028293e-03]
 [ -5.62486829e-01  -1.71407234e-03]
 [ -3.88524404e-01  -4.91640249e-03]
 [ -3.80307287e-01   3.89576006e-03]
 [ -6.04677929e-01  -1.22950748e-03]
 [ -6.71645062e-01  -2.33282684e-03]
 [ -5.65538633e-01   1.16589999e-02]
 [ -6.97547537e-01   2.69826519e-03]
 [ -6.25549356e-01  -9.03480648

np.array([state[i]]): [[-0.59858205  0.01357448]]
Target_f: [[ 0.11278705  0.21942841 -0.25535414]]
Reward[i]: -1.0
aNet: [ 0.10734179  0.21563949 -0.25276074]
np.max(aNet): 0.21563948690891266
np.amax(aNet): 0.21563948690891266
Target_f(second): [[ 0.11278705  0.21942841 -0.7865169 ]]
State: [[ -5.98582054e-01   1.35744812e-02]
 [ -5.12441897e-01  -8.62915481e-04]
 [ -3.16288133e-01   1.62174378e-03]
 [ -5.05048575e-01  -5.42345184e-03]
 [ -4.94984649e-01  -4.46823445e-03]
 [ -4.63001449e-01  -1.05489337e-02]
 [ -5.39015824e-01   1.43682488e-02]
 [ -4.16538461e-01   1.40740887e-02]
 [ -4.41848232e-01   1.18231426e-02]
 [ -5.72344229e-01  -1.44034054e-02]
 [ -5.18172269e-01   9.20261355e-04]
 [ -4.95184581e-01   6.02738003e-03]
 [ -5.40844366e-01  -3.09866751e-03]
 [ -4.91790588e-01  -5.09567176e-03]
 [ -5.11559963e-01  -1.16244270e-03]
 [ -6.08972240e-01   1.16727954e-03]
 [ -6.58636122e-01  -4.40565700e-03]
 [ -5.08553457e-01  -4.63070831e-04]
 [ -6.31221810e-01  -4.58085644e-03]
 [ 

 [ 0.08352011  0.12490682 -0.87593305]]
State: [[ -5.80974615e-01   4.15162618e-03]
 [ -5.31092781e-01  -9.56436894e-04]
 [ -4.60627677e-01   5.75109108e-03]
 [ -5.01468446e-01   5.97782772e-04]
 [ -4.44738006e-01  -1.04773165e-02]
 [ -4.70270239e-01   8.43735481e-03]
 [ -3.00562089e-01   8.80572045e-03]
 [ -5.15671717e-01   0.00000000e+00]
 [ -4.44031583e-01   5.66255438e-03]
 [ -6.47946232e-01  -7.92824451e-04]
 [ -6.83152768e-01  -8.58058037e-03]
 [ -5.43625728e-01   2.25266499e-03]
 [ -8.38955685e-01   6.44560105e-03]
 [ -4.96014504e-01   7.05364147e-03]
 [ -4.18640469e-01  -2.87122251e-03]
 [ -4.86255095e-01   7.09495044e-03]
 [ -4.84804321e-01   5.56572866e-03]
 [ -4.42172943e-01   8.80802670e-03]
 [ -5.50699692e-01  -2.40639268e-02]
 [ -5.35168011e-01  -5.16481438e-03]
 [ -6.29070144e-01  -3.64528938e-03]
 [ -5.22567685e-01   7.12417155e-03]
 [ -5.98979619e-01  -7.61325109e-03]
 [ -5.75110295e-01  -2.86691149e-03]
 [ -4.87811892e-01  -2.11076638e-04]
 [ -5.33706642e-01  -1.13830

np.array([state[i]]): [[-0.42474994  0.00513593]]
Target_f: [[ 0.02608082  0.07026697 -0.2931087 ]]
Reward[i]: -1.0
aNet: [ 0.02807596  0.06776232 -0.28762418]
np.max(aNet): 0.06776232272386551
np.amax(aNet): 0.06776232272386551
Target_f(second): [[-0.93291533  0.07026697 -0.2931087 ]]
State: [[ -4.24749944e-01   5.13592697e-03]
 [ -5.01464775e-01   6.55421768e-03]
 [ -5.86250913e-01  -5.95022879e-03]
 [ -4.74570468e-01   5.64129147e-03]
 [ -6.14587069e-01   5.88888405e-04]
 [ -4.48512757e-01  -5.68451130e-03]
 [ -4.19973116e-01  -3.59351609e-03]
 [ -4.79659728e-01  -7.28877723e-03]
 [ -4.60922030e-01  -1.41094562e-02]
 [ -5.02137296e-01  -4.44041909e-03]
 [ -4.98730126e-01   1.44955608e-03]
 [ -4.95709095e-01  -1.20991101e-02]
 [ -5.04977235e-01   5.13383027e-03]
 [ -6.02945359e-01   1.72585986e-02]
 [ -4.59839163e-01  -1.47409207e-03]
 [ -6.77328291e-01   5.38957316e-03]
 [ -6.79299847e-01  -7.48638066e-03]
 [ -4.41379385e-01   6.20073444e-03]
 [ -5.90393907e-01  -4.27772804e-03]
 [ 

...

 [-0.38569704 -0.41417894 -1.37912297]]
State: [[ -4.90370049e-01   4.81453155e-03]
 [ -3.92157950e-01  -2.74417724e-03]
 [ -5.32891061e-01  -1.97546987e-03]
 [ -4.93002104e-01  -4.05838415e-03]
 [ -5.77034339e-01  -4.94317564e-03]
 [ -5.75801860e-01  -8.06821459e-03]
 [ -5.22142947e-01  -6.39973013e-03]
 [ -6.07472034e-01  -1.20518404e-03]
 [ -3.07779895e-01  -1.63432579e-02]
 [ -5.20748062e-01   4.41812054e-03]
 [ -4.51925779e-01  -3.78949771e-04]
 [ -7.45476799e-01  -4.92116850e-03]
 [ -4.00948281e-01   9.81156312e-03]
 [ -5.63914076e-01   1.57794295e-03]
 [ -5.44218587e-01  -8.45080106e-03]
 [ -6.18133094e-01  -1.63325492e-02]
 [ -5.05600554e-01   2.15499378e-04]
 [ -4.29395921e-01  -6.20068343e-03]
 [ -5.86837477e-01   1.48236381e-03]
 [ -5.60737893e-01  -3.96828911e-05]
 [ -4.61786713e-01  -1.12220834e-02]
 [ -4.97358904e-01  -8.12821288e-03]
 [ -4.86658072e-01  -3.38696434e-03]
 [ -6.90861658e-01   4.54841383e-03]
 [ -6.05400547e-01   2.69733892e-03]
 [ -4.51756717e-01  -6.30357

np.array([state[i]]): [[ -5.64291537e-01  -1.00182975e-04]]
Target_f: [[-0.45471138 -0.50923485 -1.12582445]]
Reward[i]: -1.0
aNet: [-0.45603931 -0.5077557  -1.12692487]
np.max(aNet): -0.45603930950164795
np.amax(aNet): -0.45603930950164795
Target_f(second): [[-0.45471138 -0.50923485 -1.45147896]]
State: [[ -5.64291537e-01  -1.00182975e-04]
 [ -5.27029028e-01   2.65165552e-03]
 [ -5.41095335e-01   2.02483974e-03]
 [ -7.28722696e-01   7.58085537e-03]
 [ -5.17615003e-01   1.25132078e-03]
 [ -5.37311500e-01   4.44341189e-03]
 [ -5.17256196e-01  -6.69602390e-03]
 [ -5.97541733e-01  -1.34359684e-02]
 [ -6.26182092e-01  -1.50390328e-02]
 [ -5.44789633e-01  -4.36757870e-03]
 [ -5.13041510e-01  -1.02313771e-03]
 [ -5.77034339e-01  -4.94317564e-03]
 [ -5.85087928e-01  -2.08497258e-03]
 [ -6.33531783e-01  -1.01126806e-02]
 [ -5.35989769e-01  -1.31654489e-02]
 [ -4.14326603e-01   4.05170577e-03]
 [ -4.47728544e-01  -7.64539822e-03]
 [ -5.36025608e-01   2.92242271e-03]
 [ -5.41735130e-01  -5.13886

 [-0.40062556 -1.40023553 -0.953219  ]]
State: [[ -4.63010261e-01   5.88575284e-04]
 [ -3.00562089e-01   8.80572045e-03]
 [ -5.12774230e-01   1.89768698e-03]
 [ -4.67282028e-01   2.52731806e-03]
 [ -4.69562340e-01  -3.27935795e-03]
 [ -5.23315726e-01   4.23454785e-03]
 [ -5.88311596e-01  -7.03783348e-03]
 [ -6.17695279e-01  -4.32376299e-03]
 [ -4.53671375e-01   1.13437598e-02]
 [ -5.28009047e-01   5.38725517e-05]
 [ -5.22323627e-01  -1.23734516e-02]
 [ -4.98661592e-01   5.59032669e-03]
 [ -3.98171000e-01   1.44497713e-02]
 [ -4.02841030e-01   9.44433084e-03]
 [ -4.64048009e-01  -1.38770714e-03]
 [ -4.97651600e-01  -1.29337600e-04]
 [ -3.64876058e-01   7.30504220e-03]
 [ -5.62758388e-01   7.36002567e-03]
 [ -6.46108295e-01  -2.31704443e-02]
 [ -5.22152729e-01  -2.68345286e-04]
 [ -3.82940788e-01  -1.74863780e-03]
 [ -5.07542141e-01   3.18137676e-03]
 [ -4.36661450e-01  -4.22579254e-03]
 [ -4.22168259e-01   7.99917812e-03]
 [ -3.72181101e-01   7.40202023e-03]
 [ -5.33771759e-01   1.70336

np.array([state[i]]): [[-0.55825197  0.00172769]]
Target_f: [[-0.54669875 -0.59102631 -1.23616111]]
Reward[i]: -1.0
aNet: [-0.54470444 -0.59059358 -1.23242497]
np.max(aNet): -0.5447044372558594
np.amax(aNet): -0.5447044372558594
Target_f(second): [[-1.53925741 -0.59102631 -1.23616111]]



KeyboardInterrupt: 

# Interpretace výsledků

In [ ]:
analysis(scores, episodesList)

In [ ]:
eps = 0
for i in range(1000):
    agent.loadNN("./ki/MountainCar-v0-{}.h5" .format(eps))                                     # Nacteni vah NN
    #agent.updateTargetNet()                                                      # Nacteni NN do netTarget

    avg_score = score_estimate(env, agent, games, stateSize)
    print("Episode: {}, Kernel initializer: {}" .format(eps, avg_score))
    
    eps = eps + 50
    if eps == 1750:
        break
        
eps = 0
for i in range(1000):
    agent.loadNN("./none/MountainCar-v0-{}.h5" .format(eps))                                     # Nacteni vah NN
    #agent.updateTargetNet()                                                      # Nacteni NN do netTarget

    avg_score = score_estimate(env, agent, games, stateSize)
    print("Episode: {}, None: {}" .format(eps, avg_score))
    
    eps = eps + 50
    if eps == 2600:
        break
        
eps = 0
for i in range(1000):
    agent.loadNN("./tn-ki/MountainCar-v0-{}.h5" .format(eps))                                     # Nacteni vah NN
    #agent.updateTargetNet()                                                      # Nacteni NN do netTarget

    avg_score = score_estimate(env, agent, games, stateSize)
    print("Episode: {}, Kernel initializer, Target network: {}" .format(eps, avg_score))
    
    eps = eps + 50
    if eps == 1700:
        break
        
eps = 0
for i in range(1000):
    agent.loadNN("./tn/MountainCar-v0-{}.h5" .format(eps))                                     # Nacteni vah NN
    #agent.updateTargetNet()                                                      # Nacteni NN do netTarget

    avg_score = score_estimate(env, agent, games, stateSize)
    print("Episode: {}, Target network: {}" .format(eps, avg_score))
    
    eps = eps + 50
    if eps == 2400:
        break